In [16]:
from operator import itemgetter

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_together.embeddings import TogetherEmbeddings

from langchain.schema import format_document
from langchain_core.messages import AIMessage, HumanMessage, get_buffer_string
from langchain_core.runnables import RunnableParallel

from langchain.prompts.prompt import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.memory import ChatMessageHistory

from langchain.retrievers import MultiQueryRetriever

from helpers import get_vectorstore, save_response_to_markdown_file, read_sample

In [17]:
from models import get_together_mix, get_together_quen
ACTIVE_LLM = get_together_mix()
def get_retriever(filename, context_size = "2k"):
    model = f"togethercomputer/m2-bert-80M-{context_size}-retrieval"
    embedder = TogetherEmbeddings(model=model)
    local_vector_path = f"{filename[:-4]}-embeddings"

    vectorstore = get_vectorstore(embedder = embedder, local_vector_path = local_vector_path)
    retriever = vectorstore.as_retriever()
    return retriever

In [18]:
PDF_FILENAME = "unit6-4k.pdf"
retriever = get_retriever(PDF_FILENAME, context_size="8k")

In [19]:
# BACKGROUND_OG = "Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language."
_template = """Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:"""

CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(_template)

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
ANSWER_PROMPT = ChatPromptTemplate.from_template(template)

DEFAULT_DOCUMENT_PROMPT = PromptTemplate.from_template(template="{page_content}")

def _combine_documents(
    docs, document_prompt=DEFAULT_DOCUMENT_PROMPT, document_separator="\n\n"
):
    doc_strings = [format_document(doc, document_prompt) for doc in docs]
    return document_separator.join(doc_strings)

In [5]:
# def _combine_documents(
#     docs, document_prompt=DEFAULT_DOCUMENT_PROMPT, document_separator="\n\n"
# ):
#     doc_strings = [format_document(doc, document_prompt) for doc in docs]
#     return document_separator.join(doc_strings)

# _inputs = RunnableParallel(
#     standalone_question=RunnablePassthrough.assign(
#         chat_history=lambda x: get_buffer_string(x["chat_history"])
#     )
#     | CONDENSE_QUESTION_PROMPT
#     | ACTIVE_LLM
#     | StrOutputParser(),
# )
# _context = {
#     "context": itemgetter("standalone_question") | retriever | _combine_documents,
#     "question": lambda x: x["standalone_question"],
# }
# conversational_qa_chain = _inputs | _context | ANSWER_PROMPT | ACTIVE_LLM

In [6]:
# def get_conversational_qa_response(question: str, chat_history: list = None):
#     conversational_chat_history = chat_history
#     if conversational_chat_history is None:
#         conversational_chat_history = ChatMessageHistory()
#     response = conversational_qa_chain.invoke(
#         {
#             "question": question,
#             "chat_history": conversational_chat_history,
#         }
#     )
#     return response

# history = ChatMessageHistory()
# history.add_user_message("Only answer Yes or No. Does this paper discuss chromosomes?")
# history.add_ai_message("Yes")

# query = "Tell me more about the specific types."
# response = get_conversational_qa_response(question = query, chat_history = history.messages)

In [20]:
# Set memory
memory = ConversationBufferMemory(
    return_messages=True, input_key="question", output_key="answer"
)
# memory.save_context({"question": "Only answer Yes or No. Does this paper discuss chromosomes?"}, {"answer": "Yes"})

In [21]:
loaded_memory1 = RunnablePassthrough.assign(
    chat_history=RunnableLambda(memory.load_memory_variables) | itemgetter("history"),
)
regular_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | loaded_memory1
    | ANSWER_PROMPT
    | ACTIVE_LLM
    | StrOutputParser()
)
sample_prompt = read_sample()
response = regular_chain.invoke(sample_prompt)

PermissionDeniedError: Error code: 403 - {'error': {'message': 'Input validation error: `inputs` tokens + `max_new_tokens` must be <= 4097. Given: 3641 `inputs` tokens and 1000 `max_new_tokens`', 'type': 'invalid_request_error', 'param': 'max_tokens', 'code': None}}

In [ ]:
inputs = {"question": sample_prompt}
memory.save_context(inputs, {"answer": response["answer"].content})

In [22]:
# First we add a step to load memory
# This adds a "memory" key to the input object
loaded_memory = RunnablePassthrough.assign(
    chat_history=RunnableLambda(memory.load_memory_variables) | itemgetter("history"),
)
# Now we calculate the standalone question
standalone_question = {
    "standalone_question": {
        "question": lambda x: x["question"],
        "chat_history": lambda x: get_buffer_string(x["chat_history"]),
    }
    | CONDENSE_QUESTION_PROMPT
    | ACTIVE_LLM
    | StrOutputParser(),
}
# Now we retrieve the documents
retrieved_documents = {
    "docs": itemgetter("standalone_question") | retriever,
    "question": lambda x: x["standalone_question"],
}
# Now we construct the inputs for the final prompt
final_inputs = {
    "context": lambda x: _combine_documents(x["docs"]),
    "question": itemgetter("question"),
}
# And finally, we do the part that returns the answers
answer = {
    "answer": final_inputs | ANSWER_PROMPT | ACTIVE_LLM,
    "docs": itemgetter("docs"),
}
# And now we put it all together!
final_chain = loaded_memory | standalone_question | retrieved_documents | answer

In [23]:
# BACKUP 
# QUESTION = f"""Given the following instructions, help me create specific steps to test and adjust the retrieval threshold.
# {read_sample()}
# """
# inputs = {"question": QUESTION}
# result = final_chain.invoke(inputs)
# memory.save_context(inputs, {"answer": result["answer"].content})

In [24]:
# QUESTION = f"""Given the following instructions, help me create specific steps to test and adjust the retrieval threshold.
# {read_sample()}
# """
QUESTION = read_sample()
inputs = {"question": QUESTION}
result = final_chain.invoke(inputs)
memory.save_context(inputs, {"answer": result["answer"].content})

What are the requirements for the two text files to be submitted, and what should the functions choice-model and choice-data accomplish in the context of the given experiment?The given context does not provide explicit details about the requirements for the two text files to be submitted or the specific functions choice-model and choice-data. However, based on the general context of a paired-associate experiment and the mention of running a model and calculating correlation and mean-deviation, we can infer the following:

1. **Choice-Model**: This function likely refers to the implementation of the cognitive model, possibly using a framework like ACT-R. The model should simulate the behavior of a participant in the paired-associate experiment. It should interact with the experiment window (opened using `actr.open_exp_window`) and respond to prompts and associates, accumulating a score over trials. The model might use functions like `run-full-time/run_full_time` to run for a specific du

In [25]:
# save_response_to_markdown_file(response.content, "response.md")
save_response_to_markdown_file(result["answer"].content, "response.md")

In [12]:
def print_history_from_memory(chat_memory):
    msgs = chat_memory.buffer_as_str.split("\n")
    count = 1
    text = ""
    for msg in msgs:
        text += msg + "\n"
        count += 1
    print(text)

print_history_from_memory(memory)

Human: Creata procedure for writing an experiment which implements the choice experiment described in the assignment of the unit as well as a model which can perform the task and fits the experimental data.

You are required to submit two text files.  One named choice-model.lisp which contains your model for the task and one named either choice.lisp which contains the functions choice-model and choice-data.

The choice-model or model function must be able to present a single trial of the experiment and run a model through it.  It must have no required parameters, and there are no constraints on what it needs to return.


The choice-data function must accept a single parameter which is a number indicating how many times to run a model through the entire choice task described in unit 6.  It must collect the data from those runs and print out the average proportion of "heads" responses the model made (how many times it pressed H) in the four 12 trial groups along with the correlation and 

In [36]:
# NEW_QUESTION = f"""Given the following instructions, help me create specific steps to test and adjust the retrieval threshold.
# {read_sample()}
# """
NEW_QUESTION = read_sample()
inputs = {"question": NEW_QUESTION}
result = final_chain.invoke(inputs)
memory.save_context(inputs, {"answer": result["answer"].content})

Which navigation process best describes the scenario of a person standing at the main door to Wean, being asked to give directions from Walking to the Sky to the Tepper Quad?
Answer: Piloting. This is because the person is providing directions based on external cues and landmarks, such as the names of buildings and the layout of the campus, to help the other person navigate from one location to another. Path integration and use of a cognitive map may also be involved in the person's mental representation of the campus, but piloting is the primary process being used to provide directions in this scenario.The navigation process that best describes the scenario of a person standing at the main door to Wean, being asked to give directions from Walking to the Sky to the Tepper Quad is piloting. This is because the person is providing directions based on external cues and landmarks, such as the names of buildings and the layout of the campus, to help the other person navigate from one locati